<p id = 'toc'></p>

<h1 style = "font-family:Verdana; font-size:24px; font-weight: bold; letter-spacing: 3px; color: teal;"> TABLE OF CONTENTS </h1>

---

[<h3 style = "font-family: Verdana; font-size: 18px; text-indent: 8vw; letter-spacing: 2px; color: teal">0&nbsp;&nbsp;&nbsp;&nbsp; IMPORTS </h3>](#0)

---

<h3 style = "font-family:Verdana; font-size: 18px; text-indent: 8vw; letter-spacing: 2px; color: teal"><a href='#1'>1.&nbsp;&nbsp;&nbsp;&nbsp;DATASET</a></h3>

---

<h3 style = "font-family:Verdana; font-size: 18px; text-indent: 8vw; letter-spacing: 2px; color: teal"><a href='#2'>2.&nbsp;&nbsp;&nbsp;&nbsp;CONVERTING IMAGE DATA TO JPG</a></h3>

---

<h3 style = "font-family:Verdana; font-size: 18px; text-indent: 8vw; letter-spacing: 2px; color: teal"><a href='#3'>3.&nbsp;&nbsp;&nbsp;&nbsp;VISUALIZATIONS</a></h3>

---

<h3 style = "font-family:Verdana; font-size: 18px; text-indent: 8vw; letter-spacing: 2px; color: teal"><a href='#4'>4.&nbsp;&nbsp;&nbsp;&nbsp;DATA LOADER</a></h3>


<a id = '0'></a>
# <span><center style = "font-family: Verdana; font-size:36px; color : teal; letter-spacing: 3px; font-weight: bold">0. &nbsp;IMPORTS&nbsp;&nbsp;&nbsp;&nbsp;<a href = '#toc'>&#10514;</a></center></span>

In [30]:
# pip install pydicom

In [31]:
import pandas as pd 
import numpy as np
import os
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go


import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim

import cv2
import shutil
import random

from sklearn.preprocessing import MultiLabelBinarizer
from IPython.display import display
from PIL import Image
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.io import read_image
# from pydicom import dcmread
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)


import warnings
warnings.filterwarnings("ignore")

%config Completer.use_jedi = False

In [34]:
class config:
    TRAIN_CSV = 'train.csv'
    TEST_CSV = 'sample_submission.csv'
    TRAIN_JPG_PATH = './unifesp-xray-body-resize-270/dataset/train'
    TEST_JPG_PATH = './unifesp-xray-body-resize-270/dataset/test'
    RANDOM_SEED = 15
    resize_size = [270,270]
    img_size = [256,256]
    train_bs = 32
    val_bs = 64
    device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
    
config.device

device(type='cuda', index=0)

In [33]:
def set_seed(seed = config.RANDOM_SEED):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    print("--> SEEDING DONE")
    
    
set_seed()

--> SEEDING DONE


<a id="1"></a>

# <span><center style= "font-family:Verdana; font-size:36px; font-weight: bold; letter-spacing: 3px; color: teal">1.&nbsp;&nbsp;DATASET&nbsp;&nbsp;&nbsp;&nbsp;<a href = '#toc'>&#10514;</a> </center></span>

In [35]:
train_df = pd.read_csv(config.TRAIN_CSV)
test_df = pd.read_csv(config.TEST_CSV)
print('Traning dataframe')
display(train_df.head())
print()
print('Submission dataframe')
display(test_df.head())

Traning dataframe


,SOPInstanceUID,Target
0,1.2.826.0.1.3680043.8.498.10025629581362719970...,0
1,1.2.826.0.1.3680043.8.498.10036150326276641158...,15
2,1.2.826.0.1.3680043.8.498.10038426859954986240...,12
3,1.2.826.0.1.3680043.8.498.10050991192143676483...,14
4,1.2.826.0.1.3680043.8.498.10053309524595490852...,3



Submission dataframe


,SOPInstanceUID,Target
0,1.2.826.0.1.3680043.8.498.10001001190452685542...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
1,1.2.826.0.1.3680043.8.498.10022667601042710442...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
2,1.2.826.0.1.3680043.8.498.10024395388921105474...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
3,1.2.826.0.1.3680043.8.498.10026689165626095651...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
4,1.2.826.0.1.3680043.8.498.10035936364561920980...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...


In [37]:
def image_path_creation(path, replace_part, name_of_file_column = 'SOPInstanceUID'):
    '''
    : params
    path: directory path
    replace_part: Remove the suffix of files and save only the name of that file 

    This function iterate through the path and read all files available in that path and 
    returns dataframe containing name of file and path of that file
    
    '''

    file_names = []
    full_paths = []
    for dirpath,  dirname, filenames in os.walk(path):
        if filenames:
            for filename in filenames:
                full_path = os.path.join(dirpath, filename)
                filename = filename.replace(replace_part, '')
                full_paths.append(full_path)
                file_names.append(filename)
    image_path_dict = {name_of_file_column: file_names, 'path':full_paths}
    image_df = pd.DataFrame(image_path_dict)
    return image_df

In [38]:
image_df = image_path_creation(config.TRAIN_JPG_PATH, '.jpg')

In [40]:
image_df.SOPInstanceUID.values[:8]

array(['1.2.826.0.1.3680043.8.498.10025629581362719970278200333618114258',
       '1.2.826.0.1.3680043.8.498.10036150326276641158002573300029848125',
       '1.2.826.0.1.3680043.8.498.10038426859954986240523417641213777162',
       '1.2.826.0.1.3680043.8.498.10050991192143676483577217922037244257',
       '1.2.826.0.1.3680043.8.498.10053309524595490852815579052225336016',
       '1.2.826.0.1.3680043.8.498.10053755320637729867508668285241208441',
       '1.2.826.0.1.3680043.8.498.10062189329714053601496804394945741428',
       '1.2.826.0.1.3680043.8.498.10065930002825553435161793347987832017'],
      dtype=object)

In [28]:
train_df=pd.merge(train_df, image_df, on = 'SOPInstanceUID')

In [29]:
train_df.head(8)

,SOPInstanceUID,Target,path
0,1.2.826.0.1.3680043.8.498.10025629581362719970...,0,./unifesp-xray-body-resize-270/dataset/train\1...
1,1.2.826.0.1.3680043.8.498.10036150326276641158...,15,./unifesp-xray-body-resize-270/dataset/train\1...
2,1.2.826.0.1.3680043.8.498.10038426859954986240...,12,./unifesp-xray-body-resize-270/dataset/train\1...
3,1.2.826.0.1.3680043.8.498.10050991192143676483...,14,./unifesp-xray-body-resize-270/dataset/train\1...
4,1.2.826.0.1.3680043.8.498.10053309524595490852...,3,./unifesp-xray-body-resize-270/dataset/train\1...
5,1.2.826.0.1.3680043.8.498.10053755320637729867...,3,./unifesp-xray-body-resize-270/dataset/train\1...
6,1.2.826.0.1.3680043.8.498.10062189329714053601...,14,./unifesp-xray-body-resize-270/dataset/train\1...
7,1.2.826.0.1.3680043.8.498.10065930002825553435...,13 20,./unifesp-xray-body-resize-270/dataset/train\1...


In [ ]:
train_df.shape

<a id="2"></a>

# <span><center style = "font-family:Verdana; font-size:36px; letter-spacing:2px; font-weight:bold; color:teal">2.&nbsp;&nbsp;CONVERTING IMAGE DATA TO JPG&nbsp;&nbsp;&nbsp;&nbsp;<a href='#toc'>&#10514;</a></center></span>

In [ ]:
def read_dicom_image(path):
    dicom_file = dcmread(path)
    image = dicom_file.pixel_array
    image = image/image.max()
    return image

In [ ]:
def save_image_jpg(path, save_path_name):
    image = read_dicom_image(path)*255
    plt.imsave(save_path_name, image)

In [ ]:
os.listdir()

In [ ]:
# shutil.rmtree('./train')

In [ ]:
# Saving jpg images to working directory

# os.mkdir('./train')

# for i in tqdm(range(1000, train_df.shape[0])):
#     save_image_jpg(train_df.path[i], f'./train/{train_df.SOPInstanceUID[i]}.jpg')

In [ ]:
file_names, file_paths = image_path_creation(config.TRAIN_JPG_PATH, '.jpg')
image_df = {'SOPInstanceUID': file_names, 'path_jpg':file_paths}
image_df = pd.DataFrame(image_df)

In [ ]:
# os.listdir(config.TRAIN_JPG_PATH)

In [ ]:
image_df#, train_df.SOPInstanceUID[0]

In [ ]:
train_df_jpg=pd.merge(train_df, image_df, on = 'SOPInstanceUID')

In [ ]:
file_names, file_paths = image_path_creation(config.TEST_JPG_PATH, '.jpg')
image_df = {'SOPInstanceUID': file_names, 'path_jpg':file_paths}
image_df = pd.DataFrame(image_df)

In [ ]:
test_df_jpg=pd.merge(test_df, image_df, on = 'SOPInstanceUID')

In [ ]:
train_df_jpg.head()

In [ ]:
train_df_jpg.SOPInstanceUID[0], train_df_jpg.path_jpg[0].split('/')[-1]

image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)<a id="3"></a>

# <span><center style = "font-family:Didot; font-size:36px; letter-spacing:2px; font-weight:bold; color:teal">3.&nbsp;&nbsp;DATA EXPLORATION AND VISUALIZATIONS&nbsp;&nbsp;&nbsp;&nbsp;<a href='#toc'>&#10514;</a></center></span>

In [ ]:
train_df_jpg.Target.value_counts()

So there is Some problem in labels, lets check them out

In [ ]:
train_df_jpg.dtypes

In [ ]:
train_df_jpg['label_count']=train_df_jpg.Target.map(lambda x: len(x.split()))

In [ ]:
train_df_jpg['Target_list']=train_df_jpg.Target.map(lambda x: [int(i) for i in x.split()])

Y = train_df_jpg['Target_list']

multilabel_encoder = MultiLabelBinarizer()
Y_transformed=multilabel_encoder.fit_transform(Y)
label_columns = []

for i in range(Y_transformed.shape[1]):
    name = f'label_{i}'
    train_df_jpg[name] = Y_transformed[:,i]
    label_columns.append(name)
    
    

In [ ]:
label_columns = [column for column in train_df_jpg.columns if 'label_' in column]

In [ ]:
label_columns = label_columns[1:]

In [ ]:
train_df_jpg.head(8)

In [ ]:
fig = plt.figure(figsize=(25,15))
for i in range(15):
    ax = fig.add_subplot(3,5, i+1)
#     image = read_dicom_image(train_df.path_jpg[i*5])
    img = np.array(Image.open(train_df_jpg.path_jpg[i*5]))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    plt.imshow(img, cmap = 'gray')
    ax.set_title(train_df.Target[i*5])

In [ ]:
img = cv2.imread(train_df_jpg.path_jpg[5])
plt.imshow(img)

<a id="4"></a>

# <span><center style = "font-family:Verdana; font-size:36px; letter-spacing:2px; font-weight:bold; color:teal">3.&nbsp;&nbsp;SAVING RESIZED IMAGES&nbsp;&nbsp;&nbsp;&nbsp;<a href='#toc'>&#10514;</a></center></span>


In [ ]:
import os

In [ ]:
T = '../input/unifesp-xray-body-resize-270'
!zip -r data_file.zip ../input/unifesp-xray-body-resize-270

In [ ]:
image = cv2.imread(train_df_jpg.path_jpg[5])
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

image = cv2.resize(image, config.resize_size)

In [ ]:
config.resize_size

In [ ]:
plt.imshow(image, cmap = 'gray')

In [ ]:
# save resized images

def save_resized_images(img_path):
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, config.resize_size)
    return image

In [ ]:
# os.mkdir('./dataset')
os.mkdir('./dataset/train')
os.mkdir('./dataset/test')

In [ ]:
train_df_jpg.columns

In [ ]:
# for i in tqdm(range(train_df_jpg.shape[0])):
#     image = save_resized_images(train_df_jpg['path_jpg'][i])
#     cv2.imwrite(f'./dataset/train/{train_df_jpg.SOPInstanceUID[i]}.jpg', image)
    
for i in tqdm(range(test_df_jpg.shape[0])):
    image = save_resized_images(test_df_jpg['path_jpg'][i])
    cv2.imwrite(f'./dataset/test/{test_df_jpg.SOPInstanceUID[i]}.jpg', image)

In [ ]:
len(os.listdir('./dataset/test'))

<a id="4"></a>

# <span><center style = "font-family:Verdana; font-size:36px; letter-spacing:2px; font-weight:bold; color:teal">4.&nbsp;&nbsp;DATA LOADER&nbsp;&nbsp;&nbsp;&nbsp;<a href='#toc'>&#10514;</a></center></span>


In [ ]:
# Custom dataset

class CustomeImageDataset(Dataset):
    
    def __init__(self, data_frame, path_col, label_col, transform = None):
        self.path  = data_frame[path_col]
        self.label = data_frame[label_col]
        self.transform = transform
        
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, idx):
        img_path = self.path[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#         image = np.expand_dims(image, axis = 0)
        label = self.label.iloc[idx].to_numpy()
        label = torch.from_numpy(label)
        label = label.to(torch.float32)
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(config.resize_size),
    transforms.CenterCrop(config.img_size),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(0.3),
    transforms.ToTensor()
])

In [ ]:
config.img_size, config.resize_size

In [ ]:
image_dataset = CustomeImageDataset(train_df_jpg, 'path_jpg', label_columns, transform)

In [ ]:
# Creating dataset for training and validation=

valid_size = 0.3


num_train = len(image_dataset)
indices = list(range(num_train))
np.random.shuffle(indices)

split = int(np.floor(num_train * valid_size))
train_idx, val_idx =  indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

train_loader = DataLoader(image_dataset, batch_size = config.train_bs, 
                      sampler = train_sampler)
val_loader = DataLoader(image_dataset, batch_size = config.val_bs,
                       sampler = val_sampler)

In [ ]:
images, labels = next(iter(train_loader))

In [ ]:
images.shape, labels.shape

In [ ]:
labels.dtype

<a id="5"></a>

# <span><center style = "font-family:Verdana; font-size:36px; letter-spacing:2px; font-weight:bold; color:teal">5.&nbsp;&nbsp;MODEL BUILDING&nbsp;&nbsp;&nbsp;&nbsp;<a href='#toc'>&#10514;</a></center></span>

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size= 3, padding = 1)
        self.conv2 = nn.Conv2d(8,16, kernel_size = 3, padding = 1)
        self.conv3 = nn.Conv2d(16, 32, kernel_size = 3, padding = 1)
        self.conv4 = nn.Conv2d(32,64, kernel_size = 3, padding = 1)
        self.conv5 = nn.Conv2d(64,128, kernel_size = 3, padding = 1)
        self.conv6 = nn.Conv2d(128,144, kernel_size = 3, padding = 1)
        self.conv7 = nn.Conv2d(144,156, kernel_size = 3, padding = 1)
        self.pool = nn.MaxPool2d(2,2)
        
        self.fc1 = nn.Linear(156*2*2, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64,22)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv6(x)))
        x = self.pool(F.relu(self.conv7(x)))
        x = x.view(-1, 156*2*2)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        
        return x
        
        
        
model = Classifier()
print(model)

In [ ]:
model(images[0:1])

In [ ]:
optimizer = optim.Adam(model.parameters(), lr = 0.001)

criterion = nn.BCELoss()

In [ ]:
model.to(config.device)

In [ ]:
torch.cuda.is_available()

In [ ]:
model.cuda()

In [ ]:
torch.save(model.state_dict(), 'model_exp_save.pt')

In [ ]:
epochs = 10
training_loss = []
testing_loss = []
valid_loss_min = np.Inf

model.train()
for e in tqdm(range(epochs)):
    running_train_loss = 0
    running_test_loss = 0
    for ee, (images, labels) in enumerate(train_loader):
        model.train()
        optimizer.zero_grad()
        images, labels = images.cuda(), labels.cuda()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_train_loss += loss
            
    for images, labels in val_loader:
        model.eval()
        images, labels = images.cuda(), labels.cuda()
        output = model(images)
        loss = criterion(output, labels)
        running_test_loss += loss
    
    train_loss = running_train_loss/len(train_loader)
    valid_loss = running_test_loss/len(val_loader)
    
    print('Epoch {} \tTraining loss: {:.6f} \tValidation Loss: {:.6}'.format(
            e, train_loss, valid_loss))

    if valid_loss <=valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f})\t Saving model'.format(
                valid_loss_min, valid_loss))
        torch.save(model.state_dict(), './model_save.pt')
        valid_loss_min = valid_loss